In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_conversion import conversion
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [2]:
pd.set_option("future.no_silent_downcasting", True)

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [18]:
geno = conversion.get_geno_data()
geno = conversion.drop_single_value_cols(geno)
geno_binary = conversion.convert_geno_to_binary(geno)
geno_ternary = conversion.convert_geno_to_ternary(geno)
pheno = conversion.get_pheno_data()
#geno_ternary_filled = conversion.fill_nan_with_distribution(geno_ternary.astype(float))
df = pd.concat([geno_ternary.astype(float), pheno], axis=1)

In [19]:
# Convert Sex to binary: F = 1 and M = 0

df['Sex'] = df['Sex'] == 'F'
df['Sex'] = df['Sex'].astype(float)

# Create separate dataframes for female and male

df_f = df[df['Sex'] == 1]
df_m = df[df['Sex'] == 0]

# Fill NaNs

df_f1 = conversion.fill_nan_with_distribution(df_f.copy())
df_m1 = conversion.fill_nan_with_distribution(df_m.copy())

# Drop single value columns

df_female=conversion.drop_single_value_cols(df_f1)
df_male = conversion.drop_single_value_cols(df_m1)

In [20]:
# Add intersept column to the dataframe

intercept = pd.DataFrame({'intercept': np.ones(df_female.shape[0])})
intercept.set_index(df_female.index, inplace= True)
df_female = pd.concat([intercept, df_female], axis=1)

intercept = pd.DataFrame({'intercept': np.ones(df_male.shape[0])})
intercept.set_index(df_male.index, inplace= True)
df_male = pd.concat([intercept, df_male], axis=1)

In [21]:
# Get female/male SNPs

snp_female = df_female.columns[1:-3]
snp_male = df_male.columns[1:-3]

print('The number of female snps: ',len(snp_female))
print('The number of male snps: ', len(snp_male))

The number of female snps:  6615
The number of male snps:  6714


In [22]:
# Record p-values for df_female as a dictionary with {SNP : p-value}

p_values_female = {}
for column in snp_female:
    model = sm.OLS(df_female.NEUT,df_female[['intercept',column]],missing = 'drop')
    res = model.fit()
    p_values_female[column] = res.pvalues.iloc[1]

In [9]:
# Sorted p_values dict
sorted_p_values_female = dict(sorted(p_values_female.items(), key=lambda item: item[1]))

In [26]:
# Train test splits for both dataframes

df_female_train, df_female_test = train_test_split(df_female.copy(), test_size=0.3, random_state=402, shuffle = True)
df_male_train, df_male_test = train_test_split(df_male.copy(), test_size=0.3, random_state=402, shuffle = True)



In [27]:
## Import KFold
from sklearn.model_selection import KFold

In [283]:
## Make kfold object
kfold = KFold(n_splits=5, 
              shuffle=True, 
              random_state = 403)

In [145]:
## make empty mse holder
cv_baseline_mses = np.zeros(5)


## loop through all splits
i = 0
for train_index, test_index in kfold.split(df_female):
    ## get train and holdout sets
    df_female_train_train = df_female.iloc[train_index]
    df_female_holdout = df_female.iloc[test_index]

    ## make clone
    linreg = LinearRegression()
        
    ## fit clone
    linreg.fit(df_female_train_train[['intercept']+ list(snp_female)], y=df_female_train_train['NEUT'])
    linear_coefs = pd.Series(linreg.coef_, index=['intercept']+ list(snp_female))
    y_pred=linreg.predict(df_female_test[['intercept']+ list(snp_female)])
    y_true = df_female_test.NEUT

    ## record mse
    cv_baseline_mses[i] = mean_squared_error(y_true,y_pred)
    i=i+1  

In [146]:
cv_baseline_mses.mean()

np.float64(39604.228742224004)

In [147]:
models = [list(sorted_p_values_female.keys())[0:5*i] for i in range(200,400)]

## make empty mse holder
cv_mses = np.zeros((5, len(models)))


## loop through all splits
i = 0
for train_index, test_index in kfold.split(df_female):
    ## get train and holdout sets
    df_female_train_train = df_female.iloc[train_index]
    df_female_holdout = df_female.iloc[test_index]


    ## loop through all models
    j = 0
    for model in models:
        ## make clone
        linreg = LinearRegression()
        
        ## fit clone
        linreg.fit(df_female_train_train[['intercept']+ model], y=df_female_train_train['NEUT'])
        linear_coefs = pd.Series(linreg.coef_, index=['intercept']+ model)
        y_pred=linreg.predict(df_female_test[['intercept']+ model])
        y_true = df_female_test.NEUT

        ## record mse
        cv_mses[i,j] = mean_squared_error(y_true,y_pred)
        j=j+1
    i=i+1

In [148]:
## Which one had the smallest avg cv mse?
np.argmin(np.mean(cv_mses, axis=0))

np.int64(195)

In [149]:
## which model had lowest mean mse?
print("The model with lowest mean cv mse included the features", 
      len(models[np.argmin(np.mean(cv_mses, axis=0))]),
      "and had an avg cv mse of",
      np.mean(cv_mses, axis=0)[np.argmin(np.mean(cv_mses, axis=0))])

The model with lowest mean cv mse included the features 1975 and had an avg cv mse of 33492.02154046813


In [153]:
cv_mses[:,193].mean()

np.float64(33996.60223190622)

## A lasso reminder

The other algorithmic approach we will touch on is lasso regression.

Recall from our Regularization that lasso can be used for feature selection by slowly increasing the value of the hyperparameter $\alpha$ and observing the persistence of the coefficients (i.e. which coefficients stay above $0$ the longest?). 


In [253]:
alphas = [0.000001,.00001,.0001,.001,.01,.1,.25,.5,.75,1,5,10,25,50,100,1000,10000]
features = list(sorted_p_values_female.keys())

## make coefficient holder
coefs = np.zeros((len(alphas), len(features)))


## Loop through alphas
for i in range(len(alphas)):
    ## make lasso model
    lasso = Lasso(alpha=alphas[i], max_iter=10000000)
    
    
    ## fit model
    lasso.fit(df_female_train[features], df_female_train.NEUT.values)
    
    ## record coefs
    coefs[i,:] = lasso.coef_

    ## make prediction
    y_pred = lasso.predict(df_female_test[features])
    y_true = df_female_test.NEUT

    ## compute mse
    mse_lasso = mean_squared_error(y_true,y_pred)


In [254]:
mse_lasso

np.float64(125539.98881839996)

In [241]:
## use dataframe to look at coefs

df = pd.DataFrame(data=coefs, index=alphas, columns=features)
df

,JAX00291370,backupUNC170333568,UNC060024140,JAX00504321,backupJAX00265198,UNC_rs33539152,JAX00382788,UNC140311549,UNC190011937,UNC_rs51740727,...,JAX00204570,JAX00155945,JAX00518172,UNC070619927,backupJAX00110129,UNC200106288,JAX00426011,backupUNC030615251,UNC010637850,JAX00185134
0.000001,-186.928337,123.487950,-155.886193,110.469259,-103.785750,-85.864839,79.916108,-64.825275,45.231895,7.751017,...,0.135092,-0.200305,-0.264018,0.001342,-0.011015,0.048106,-0.148554,-0.241179,-0.119658,0.001879
0.000010,-186.928144,123.487998,-155.886058,110.469182,-103.785731,-85.864865,79.916081,-64.825225,45.231938,7.750877,...,0.134959,-0.200236,-0.263989,0.001226,-0.011029,0.047975,-0.148529,-0.241078,-0.119692,0.001933
0.000100,-186.807135,123.476835,-155.707873,110.365923,-103.760325,-85.842780,79.964916,-64.771711,45.302829,7.598870,...,0.000692,-0.172087,-0.250301,-0.000000,-0.000000,-0.006326,-0.182358,-0.151032,0.000000,0.000000
0.001000,-174.930176,116.110637,-129.502437,101.002752,-93.772004,-90.303779,60.111408,-50.831612,45.043065,-0.000000,...,0.000000,-0.000000,-0.000000,-1.466360,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000
0.010000,-36.043518,32.912146,-0.000000,0.000000,-16.952163,-78.661127,0.000000,-0.000000,0.000000,-0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000
0.100000,-6.627638,18.439391,-0.000000,0.000000,-20.711984,-66.308187,0.000000,-0.000000,0.000000,-0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000
0.250000,-9.942237,19.471708,-0.000000,0.000000,-19.868875,-66.985337,0.000000,-0.000000,0.000000,-0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000
0.500000,-15.504612,18.660160,-0.000000,0.000000,-16.931407,-67.391354,0.000000,-0.000000,0.000000,-0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000
0.750000,-22.369540,16.833785,-0.000000,0.000000,-18.902603,-68.910825,0.000000,-0.000000,0.000000,-0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000
1.000000,-31.930492,15.039977,-0.000000,0.000000,-19.617853,-71.082190,0.000000,-0.000000,0.000000,-0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000


In [242]:
# This shows the first value of alpha where the parameter for the feature was set to zero.

df.ne(0).idxmin().sort_values(ascending=False)

backupUNC170333568    100.000000
UNC190055214          100.000000
JAX00002267           100.000000
UNC020442578          100.000000
UNC070359148          100.000000
                         ...    
UNC030069565            0.000001
UNC030407003            0.000001
UNC100047554            0.000001
UNC010475009            0.000001
UNC030419416            0.000001
Length: 5000, dtype: float64

In [257]:
sorted_lasso_features = list(df.ne(0).idxmin().sort_values(ascending=False).index)

In [293]:
models = [sorted_lasso_features[0:i] for i in range(1500,2000)]

## make empty mse holder
cv_mses = np.zeros((5, len(models)))


## loop through all splits
i = 0
for train_index, test_index in kfold.split(df_female):
    ## get train and holdout sets
    df_female_train_train = df_female.iloc[train_index]
    df_female_holdout = df_female.iloc[test_index]


    ## loop through all models
    j = 0
    for model in models:
        ## make clone
        linreg = LinearRegression()
        
        ## fit clone
        linreg.fit(df_female_train_train[['intercept']+ model], y=df_female_train_train['NEUT'])
        linear_coefs = pd.Series(linreg.coef_, index=['intercept']+ model)
        y_pred=linreg.predict(df_female_test[['intercept']+ model])
        y_true = df_female_test.NEUT

        ## record mse
        cv_mses[i,j] = mean_squared_error(y_true,y_pred)
        j=j+1
    i=i+1

In [294]:
## Which one had the smallest avg cv mse?
np.argmin(np.mean(cv_mses, axis=0))

np.int64(70)

In [295]:
## which model had lowest mean mse?
print("The model with lowest mean cv mse included the features", 
      len(models[np.argmin(np.mean(cv_mses, axis=0))]),
      "and had an avg cv mse of",
      np.mean(cv_mses, axis=0)[np.argmin(np.mean(cv_mses, axis=0))])

The model with lowest mean cv mse included the features 1570 and had an avg cv mse of 31036.328116945013
